In [1]:
library(edgeR)
library(limma)
library(Glimma)
library(ggplot2)
library(RColorBrewer)

Loading required package: limma



In [82]:
# Reading L3 count ordered data
allRawL3CountsData <- read.csv("/home/jupyter/clean_csv/raw_L3counts_data/raw_L3counts__ordered_data.csv", row.names=1)

In [83]:
# Reading groups like Sex, Ethnicity, etc
groupsList <- read.csv("/home/jupyter/clean_csv/raw_L3counts_data/transcript_data_groups.csv", row.names=1)
sex <- factor(groupsList$Sex)
Ethnicity <- factor(groupsList$Ethnicity)
Race <- factor(groupsList$Race)
SiteOnset <- factor(groupsList$SiteOnset)
age <- factor(groupsList$Age)

In [84]:
#Calculate count per million and keep samples
myCPM <- cpm(allRawL3CountsData)
thresh <- myCPM > 0.5
keep <- rowSums(thresh) >= 2
counts.keep <- allRawL3CountsData[keep,]
dge <- DGEList(counts=counts.keep)

In [85]:
#plot the library size per sample
#barplot(dge$samples$lib.size, names=colnames(dge), las=2)

In [86]:
nor.data <-normalizeQuantiles(counts.keep, ties=TRUE)
dge <- DGEList(nor.data)
#heatmap(as.matrix(nor.data))

In [87]:
write.csv(nor.data, "/home/jupyter/clean_csv/raw_L3counts_data/quantile_normalization.csv")

In [89]:
# To perform quasi-likelihood F-tests:
design <- model.matrix(~SiteOnset + Race + sex + age)
y <- estimateDisp(dge,design)
fit <- glmQLFit(y,design) 
qlf <- glmQLFTest(fit,coef=2)
qlf <- topTags(qlf, n=length(qlf$table$logFC))
FDRFilterQLF <- qlf$table$FDR <= 0.05
qlfWithFDRFilter <- qlf$table[FDRFilterQLF, ]
qlfWithFDRFilter


,logFC,logCPM,F,PValue,FDR
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000200680,9.751148,1.8792263,36.94066,9.569169e-09,0.0002382723
ENSG00000234287,-4.220507,-1.3562556,30.82328,1.245414e-07,0.0011201162
ENSG00000200726,8.701960,0.8588241,30.63560,1.349538e-07,0.0011201162
ENSG00000200503,10.266705,2.0844576,29.92843,1.827912e-07,0.0011378751
ENSG00000198744,-6.870285,1.3016574,28.01961,4.175360e-07,0.0020793293
ENSG00000144451,0.968775,4.6602981,23.65345,2.876880e-06,0.0119390521
ENSG00000167332,-5.009756,-2.7501258,21.81509,6.602643e-06,0.0234865459
ENSG00000200812,5.147884,0.9125128,20.02739,1.497586e-05,0.0466123779


In [92]:
getwd()

[1] "/home/jovyan"

In [90]:
#To perform likelihood ratio tests:
fit2 <- glmFit(y,design)
lrt <- glmLRT(fit2,coef=2)
lrt <- topTags(lrt, n=length(lrt$table$logFC))
FDRFilterLTR <- lrt$table$FDR <= 0.05
lrtWithFDRFilter <- lrt$table[FDRFilterLTR, ]

lrtWithFDRFilter

,logFC,logCPM,LR,PValue,FDR
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000200680,9.7516096,1.8792263,34.40955,4.465280e-09,0.0001111855
ENSG00000198744,-6.8913607,1.3016574,32.87564,9.824643e-09,0.0001223168
ENSG00000200503,10.2665476,2.0844576,30.99738,2.583767e-08,0.0002144526
ENSG00000234287,-4.2329863,-1.3562556,29.39691,5.897160e-08,0.0003670982
ENSG00000200726,8.7021839,0.8588241,28.20598,1.090673e-07,0.0005431552
ENSG00000144451,0.9688244,4.6602981,21.37180,3.782951e-06,0.0156992456
ENSG00000200812,5.1483550,0.9125128,19.99669,7.757619e-06,0.0275949577
